# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import datetime as dt
import stumpy
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("stumpy version: " + stumpy.__version__)

numpy version: 1.18.1
pandas version: 1.0.1
stumpy version: 1.7.2


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup cyclical_business_process.csv<br>
| eval _time=strptime(_time, "%Y-%m-%dT%H:%M:%S")<br>
| timechart span=15m avg(logons) as logons<br>
| fit MLTKContainer mode=stage algo=stumpy logons from _time into app:stumpy_anomalies<br>


After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [36]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [37]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("stumpy_anomalies")
print(df.describe())
print(param)

              _time       logons
count  3.681000e+03  3681.000000
mean   1.534990e+09    55.685511
std    9.564817e+05    42.976066
min    1.533334e+09     0.666667
25%    1.534162e+09    18.666667
50%    1.534990e+09    43.333333
75%    1.535818e+09    85.000000
max    1.536646e+09   245.266667
{'options': {'params': {'mode': 'stage', 'algo': 'stumpy'}, 'args': ['logons', '_time'], 'target_variable': ['logons'], 'feature_variables': ['_time'], 'model_name': 'stumpy_anomalies', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['_time'], 'target_variables': ['logons']}


## Stage 2 - create and initialize a model

In [38]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    return model

In [39]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{}


## Stage 3 - fit the model

In [40]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model created"}
    return info

In [41]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model created'}


## Stage 4 - apply the model

In [42]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    m = 24
    if 'options' in param:
        if 'params' in param['options']:
            if 'm' in param['options']['params']:
                m = int(param['options']['params']['m'])
    target = param['target_variables'][0]
    mp = stumpy.stump(df[target], m)    
    result = pd.DataFrame(mp[:, 0], columns=['matrix_profile'])
    return pd.concat([df, result], axis=1)

In [43]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

           _time     logons matrix_profile
0     1533333600   0.666667        1.32494
1     1533334500   1.233333         1.6778
2     1533335400   2.000000        1.95066
3     1533336300   1.233333        2.03543
4     1533337200   3.000000        2.05146
...          ...        ...            ...
3676  1536642000  50.000000            NaN
3677  1536642900  50.666667            NaN
3678  1536643800  47.333333            NaN
3679  1536644700  58.533333            NaN
3680  1536645600  17.100000            NaN

[3681 rows x 3 columns]


## Stage 5 - save the model

In [33]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [34]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [35]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code